In [5]:
import h5py
import json
import numpy as np
from numpy import argwhere

# Open the dataset
path = '/media/wjyun/08EA7DEDEA7DD800/Radioml2018/'
hdf5_file = h5py.File(str(path)+"GOLD_XYZ_OSC.0001_1024.hdf5",  'r')
# Load the modulation classes. You can also copy and paste the content of classes-fixed.txt.
modulation_classes = json.load(open(path+"classes-fixed.json", 'r'))

# Read the HDF5 groups
data = hdf5_file['X']
modulation_onehot = hdf5_file['Y']
snr = hdf5_file['Z']



In [6]:
data_reshape = np.transpose(data,[0,2,1])
data_reshape.shape

(2555904, 2, 1024)

In [11]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import column_or_1d

class MyLabelEncoder(LabelEncoder):

    def fit(self, y):
        y = column_or_1d(y, warn=True)
        self.classes_ = pd.Series(y).unique()
        return self

encoder = MyLabelEncoder()
encoder.fit(modulation_classes)

MyLabelEncoder()

In [14]:
# Extract SNR 30
pos = np.where(np.array(snr) == 30)[0]
temp_data = data_reshape[pos]
temp_class = modulation_onehot[pos]
numeric_class = np.argmax(temp_class,axis=1)
char_class = encoder.inverse_transform(numeric_class)
print(char_class[4096], char_class.shape)


4ASK (98304,)


In [21]:
hf = h5py.File('./datas/2018_SNR_30.hdf5', 'w')
test1 = hf.create_dataset('data', data=temp_data)
test2 = hf.create_dataset('label', data=char_class)
hf.close

<bound method File.close of <HDF5 file "2018_SNR_30.hdf5" (mode r+)>>

In [30]:
test3 = h5py.File('./datas/2018_SNR_30.hdf5', 'r')
test3['data'].shape

(98304, 2, 1024)